In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [92]:
be = pd.read_csv('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/BA_CODEX/quantification/BE_tonsil/processed/BE_Tonsil_l3_dryad_cleaned.csv')

In [93]:
be

,CHGA,PDL1,CD56,CK7,FoxP3,CD21,MUC1,PD1,CD11b,CD4,...,Arginase1,CD73,CD206,MUC6,COX2,MMP9,x,y,sample_name,cell_type
0,-0.015838,-0.262392,-0.328807,-0.030104,-0.169786,-0.195315,-0.144450,-0.314147,-0.306023,-0.681588,...,0.183429,-0.471492,0.368175,-0.137850,0.148542,-0.161118,394.0,3516.0,tonsil,Innate
1,-0.063117,-0.075990,1.852914,-0.030104,-0.169786,2.408519,-0.144450,1.413897,1.150271,-0.499367,...,3.152803,-0.460310,0.406059,0.247144,0.549028,-0.114117,5469.0,2463.0,tonsil,Innate
2,-0.070705,-0.262392,-0.362758,-0.030104,-0.169786,0.281066,-0.144449,-0.270863,-0.306020,-0.681588,...,-0.300006,0.798789,-0.238657,-0.137942,-0.167777,-0.161118,2506.0,4793.0,tonsil,PDPN
3,-0.061208,-0.186271,-0.074156,-0.030104,-0.169786,-0.078391,-0.144447,-0.088802,0.294455,-0.681588,...,1.068848,-0.256614,-0.230057,-0.130576,-0.069935,-0.161118,8933.0,796.0,tonsil,Endothelial
4,-0.070660,-0.123091,0.006811,-0.030104,0.100844,0.054867,-0.144450,-0.046886,2.337389,-0.588588,...,3.108413,-0.451207,-0.165934,-0.126458,0.048613,-0.153662,4793.0,1885.0,tonsil,Innate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219921,-0.070250,-0.262392,0.637652,-0.030104,-0.169786,-0.174148,-0.144450,0.044370,-0.306023,-0.572092,...,-0.263463,-0.367291,-0.110060,-0.137930,-0.096468,-0.161118,4373.0,1739.0,Barretts Esophagus,Innate
219922,-0.058163,-0.160154,0.202644,-0.030104,-0.169786,-0.219187,-0.144450,-0.201268,-0.306023,-0.619760,...,-0.193195,0.916953,3.677105,-0.137942,-0.163933,-0.161118,4831.0,2782.0,Barretts Esophagus,Innate
219923,-0.067054,-0.214624,0.057478,-0.004263,-0.166512,-0.155890,0.100125,-0.122829,-0.301920,-0.255616,...,-0.041300,-0.163893,-0.192927,-0.072235,-0.137067,-0.140883,1592.0,8542.0,Barretts Esophagus,Innate
219924,-0.013725,-0.138051,-0.057624,-0.030104,-0.169778,-0.309653,-0.121936,-0.334723,-0.306023,0.712992,...,-0.651138,-0.471492,-0.249438,-0.137942,-0.171432,-0.161118,3236.0,238.0,Barretts Esophagus,Innate


In [94]:
tonsil = be[be['sample_name']=='tonsil']
be = be[be['sample_name']=='Barretts Esophagus']

In [95]:
tonsil

,CHGA,PDL1,CD56,CK7,FoxP3,CD21,MUC1,PD1,CD11b,CD4,...,Arginase1,CD73,CD206,MUC6,COX2,MMP9,x,y,sample_name,cell_type
0,-0.015838,-0.262392,-0.328807,-0.030104,-0.169786,-0.195315,-0.144450,-0.314147,-0.306023,-0.681588,...,0.183429,-0.471492,0.368175,-0.137850,0.148542,-0.161118,394.0,3516.0,tonsil,Innate
1,-0.063117,-0.075990,1.852914,-0.030104,-0.169786,2.408519,-0.144450,1.413897,1.150271,-0.499367,...,3.152803,-0.460310,0.406059,0.247144,0.549028,-0.114117,5469.0,2463.0,tonsil,Innate
2,-0.070705,-0.262392,-0.362758,-0.030104,-0.169786,0.281066,-0.144449,-0.270863,-0.306020,-0.681588,...,-0.300006,0.798789,-0.238657,-0.137942,-0.167777,-0.161118,2506.0,4793.0,tonsil,PDPN
3,-0.061208,-0.186271,-0.074156,-0.030104,-0.169786,-0.078391,-0.144447,-0.088802,0.294455,-0.681588,...,1.068848,-0.256614,-0.230057,-0.130576,-0.069935,-0.161118,8933.0,796.0,tonsil,Endothelial
4,-0.070660,-0.123091,0.006811,-0.030104,0.100844,0.054867,-0.144450,-0.046886,2.337389,-0.588588,...,3.108413,-0.451207,-0.165934,-0.126458,0.048613,-0.153662,4793.0,1885.0,tonsil,Innate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173963,-0.041959,-0.126433,-0.283366,-0.021678,-0.163694,-0.147510,-0.124578,-0.107717,-0.263219,-0.432906,...,-0.080908,-0.408275,-0.223434,-0.098247,-0.111970,-0.161118,8248.0,6715.0,tonsil,Squamous_epithelial
173964,-0.047270,-0.012078,0.711355,-0.030104,-0.162891,-0.026921,-0.059392,-0.167543,-0.305407,-0.562331,...,-0.187506,-0.308462,-0.225225,-0.120709,-0.121955,-0.161118,8454.0,6490.0,tonsil,Squamous_epithelial
173965,-0.070705,-0.262392,-0.284593,-0.030104,-0.169786,-0.208046,-0.126339,-0.260718,-0.306023,-0.660122,...,-0.076339,-0.396891,-0.236290,-0.135009,-0.132492,-0.161118,6318.0,4991.0,tonsil,Squamous_epithelial
173966,-0.062836,1.355875,0.139548,-0.030104,-0.169739,-0.246346,-0.110626,-0.267340,-0.305779,-0.467839,...,-0.173762,-0.460017,-0.244242,-0.127680,0.124011,-0.161097,7635.0,7871.0,tonsil,Squamous_epithelial


In [96]:
be

,CHGA,PDL1,CD56,CK7,FoxP3,CD21,MUC1,PD1,CD11b,CD4,...,Arginase1,CD73,CD206,MUC6,COX2,MMP9,x,y,sample_name,cell_type
173968,-0.070705,-0.262392,-0.501648,-0.030104,-0.169786,-0.243261,-0.144450,-0.384596,-0.305905,-0.570345,...,0.269265,-0.471492,-0.249438,-0.137942,-0.174521,-0.161118,7298.0,931.0,Barretts Esophagus,Innate
173969,-0.070705,-0.262392,0.828747,-0.030104,-0.169786,-0.217584,-0.144450,-0.255562,-0.306023,-0.681588,...,-0.500090,-0.471492,-0.249438,-0.137942,0.204625,-0.161118,5387.0,2426.0,Barretts Esophagus,Endothelial
173970,-0.070705,-0.262392,-0.084720,-0.030104,-0.169686,-0.172772,-0.144450,-0.262302,-0.306023,-0.640059,...,-0.651138,-0.471492,-0.242207,-0.136360,0.078458,-0.161118,9327.0,2790.0,Barretts Esophagus,Endothelial
173971,-0.020742,-0.214068,-0.428835,-0.030104,-0.169786,-0.232313,-0.144450,-0.351660,-0.306023,-0.681588,...,-0.651138,-0.378619,-0.247338,-0.137942,-0.174521,-0.161118,29.0,5294.0,Barretts Esophagus,Endothelial
173972,-0.064866,-0.233875,-0.401097,-0.030104,-0.166753,-0.163125,-0.144450,-0.272000,-0.306023,-0.681588,...,-0.478014,-0.471492,-0.249438,-0.137942,-0.174521,-0.161118,6539.0,8353.0,Barretts Esophagus,Endothelial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219921,-0.070250,-0.262392,0.637652,-0.030104,-0.169786,-0.174148,-0.144450,0.044370,-0.306023,-0.572092,...,-0.263463,-0.367291,-0.110060,-0.137930,-0.096468,-0.161118,4373.0,1739.0,Barretts Esophagus,Innate
219922,-0.058163,-0.160154,0.202644,-0.030104,-0.169786,-0.219187,-0.144450,-0.201268,-0.306023,-0.619760,...,-0.193195,0.916953,3.677105,-0.137942,-0.163933,-0.161118,4831.0,2782.0,Barretts Esophagus,Innate
219923,-0.067054,-0.214624,0.057478,-0.004263,-0.166512,-0.155890,0.100125,-0.122829,-0.301920,-0.255616,...,-0.041300,-0.163893,-0.192927,-0.072235,-0.137067,-0.140883,1592.0,8542.0,Barretts Esophagus,Innate
219924,-0.013725,-0.138051,-0.057624,-0.030104,-0.169778,-0.309653,-0.121936,-0.334723,-0.306023,0.712992,...,-0.651138,-0.471492,-0.249438,-0.137942,-0.171432,-0.161118,3236.0,238.0,Barretts Esophagus,Innate


In [97]:
be['cell_type'].unique()

array(['Innate', 'Endothelial', 'PDPN', 'T', 'Glandular_epi',
       'Secretory_epithelial', 'SmoothMuscle', 'Squamous_epithelial',
       'Paneth', 'Nerve', 'Stroma', 'Plasma'], dtype=object)

In [98]:
tonsil['cell_type'].unique()

array(['Innate', 'PDPN', 'Endothelial', 'B', 'T', 'Squamous_epithelial',
       'Stroma', 'SmoothMuscle', 'Plasma', 'Nerve'], dtype=object)

In [99]:
matching_celltypes = [x for x in tonsil['cell_type'].unique() if x in be['cell_type'].unique()]

In [100]:
matching_celltypes

['Innate',
 'PDPN',
 'Endothelial',
 'T',
 'Squamous_epithelial',
 'Stroma',
 'SmoothMuscle',
 'Plasma',
 'Nerve']

In [101]:
tonsil_filtered = tonsil[tonsil['cell_type'].isin(matching_celltypes)]
be_filtered = be[be['cell_type'].isin(matching_celltypes)]

In [102]:
tonsil_filtered

,CHGA,PDL1,CD56,CK7,FoxP3,CD21,MUC1,PD1,CD11b,CD4,...,Arginase1,CD73,CD206,MUC6,COX2,MMP9,x,y,sample_name,cell_type
0,-0.015838,-0.262392,-0.328807,-0.030104,-0.169786,-0.195315,-0.144450,-0.314147,-0.306023,-0.681588,...,0.183429,-0.471492,0.368175,-0.137850,0.148542,-0.161118,394.0,3516.0,tonsil,Innate
1,-0.063117,-0.075990,1.852914,-0.030104,-0.169786,2.408519,-0.144450,1.413897,1.150271,-0.499367,...,3.152803,-0.460310,0.406059,0.247144,0.549028,-0.114117,5469.0,2463.0,tonsil,Innate
2,-0.070705,-0.262392,-0.362758,-0.030104,-0.169786,0.281066,-0.144449,-0.270863,-0.306020,-0.681588,...,-0.300006,0.798789,-0.238657,-0.137942,-0.167777,-0.161118,2506.0,4793.0,tonsil,PDPN
3,-0.061208,-0.186271,-0.074156,-0.030104,-0.169786,-0.078391,-0.144447,-0.088802,0.294455,-0.681588,...,1.068848,-0.256614,-0.230057,-0.130576,-0.069935,-0.161118,8933.0,796.0,tonsil,Endothelial
4,-0.070660,-0.123091,0.006811,-0.030104,0.100844,0.054867,-0.144450,-0.046886,2.337389,-0.588588,...,3.108413,-0.451207,-0.165934,-0.126458,0.048613,-0.153662,4793.0,1885.0,tonsil,Innate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173963,-0.041959,-0.126433,-0.283366,-0.021678,-0.163694,-0.147510,-0.124578,-0.107717,-0.263219,-0.432906,...,-0.080908,-0.408275,-0.223434,-0.098247,-0.111970,-0.161118,8248.0,6715.0,tonsil,Squamous_epithelial
173964,-0.047270,-0.012078,0.711355,-0.030104,-0.162891,-0.026921,-0.059392,-0.167543,-0.305407,-0.562331,...,-0.187506,-0.308462,-0.225225,-0.120709,-0.121955,-0.161118,8454.0,6490.0,tonsil,Squamous_epithelial
173965,-0.070705,-0.262392,-0.284593,-0.030104,-0.169786,-0.208046,-0.126339,-0.260718,-0.306023,-0.660122,...,-0.076339,-0.396891,-0.236290,-0.135009,-0.132492,-0.161118,6318.0,4991.0,tonsil,Squamous_epithelial
173966,-0.062836,1.355875,0.139548,-0.030104,-0.169739,-0.246346,-0.110626,-0.267340,-0.305779,-0.467839,...,-0.173762,-0.460017,-0.244242,-0.127680,0.124011,-0.161097,7635.0,7871.0,tonsil,Squamous_epithelial


In [103]:
be_filtered

,CHGA,PDL1,CD56,CK7,FoxP3,CD21,MUC1,PD1,CD11b,CD4,...,Arginase1,CD73,CD206,MUC6,COX2,MMP9,x,y,sample_name,cell_type
173968,-0.070705,-0.262392,-0.501648,-0.030104,-0.169786,-0.243261,-0.144450,-0.384596,-0.305905,-0.570345,...,0.269265,-0.471492,-0.249438,-0.137942,-0.174521,-0.161118,7298.0,931.0,Barretts Esophagus,Innate
173969,-0.070705,-0.262392,0.828747,-0.030104,-0.169786,-0.217584,-0.144450,-0.255562,-0.306023,-0.681588,...,-0.500090,-0.471492,-0.249438,-0.137942,0.204625,-0.161118,5387.0,2426.0,Barretts Esophagus,Endothelial
173970,-0.070705,-0.262392,-0.084720,-0.030104,-0.169686,-0.172772,-0.144450,-0.262302,-0.306023,-0.640059,...,-0.651138,-0.471492,-0.242207,-0.136360,0.078458,-0.161118,9327.0,2790.0,Barretts Esophagus,Endothelial
173971,-0.020742,-0.214068,-0.428835,-0.030104,-0.169786,-0.232313,-0.144450,-0.351660,-0.306023,-0.681588,...,-0.651138,-0.378619,-0.247338,-0.137942,-0.174521,-0.161118,29.0,5294.0,Barretts Esophagus,Endothelial
173972,-0.064866,-0.233875,-0.401097,-0.030104,-0.166753,-0.163125,-0.144450,-0.272000,-0.306023,-0.681588,...,-0.478014,-0.471492,-0.249438,-0.137942,-0.174521,-0.161118,6539.0,8353.0,Barretts Esophagus,Endothelial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219921,-0.070250,-0.262392,0.637652,-0.030104,-0.169786,-0.174148,-0.144450,0.044370,-0.306023,-0.572092,...,-0.263463,-0.367291,-0.110060,-0.137930,-0.096468,-0.161118,4373.0,1739.0,Barretts Esophagus,Innate
219922,-0.058163,-0.160154,0.202644,-0.030104,-0.169786,-0.219187,-0.144450,-0.201268,-0.306023,-0.619760,...,-0.193195,0.916953,3.677105,-0.137942,-0.163933,-0.161118,4831.0,2782.0,Barretts Esophagus,Innate
219923,-0.067054,-0.214624,0.057478,-0.004263,-0.166512,-0.155890,0.100125,-0.122829,-0.301920,-0.255616,...,-0.041300,-0.163893,-0.192927,-0.072235,-0.137067,-0.140883,1592.0,8542.0,Barretts Esophagus,Innate
219924,-0.013725,-0.138051,-0.057624,-0.030104,-0.169778,-0.309653,-0.121936,-0.334723,-0.306023,0.712992,...,-0.651138,-0.471492,-0.249438,-0.137942,-0.171432,-0.161118,3236.0,238.0,Barretts Esophagus,Innate


In [104]:
tonsil_filtered.drop(columns=['sample_name', 'x', 'y'], inplace=True)
be_filtered.drop(columns=['sample_name', 'x', 'y'], inplace=True)

/var/folders/ry/jl4ll0zx1l709rh4r7_0cq6r0000gn/T/ipykernel_17834/1778666527.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tonsil_filtered.drop(columns=['sample_name', 'x', 'y'], inplace=True)
/var/folders/ry/jl4ll0zx1l709rh4r7_0cq6r0000gn/T/ipykernel_17834/1778666527.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  be_filtered.drop(columns=['sample_name', 'x', 'y'], inplace=True)


In [105]:
tonsil_filtered

,CHGA,PDL1,CD56,CK7,FoxP3,CD21,MUC1,PD1,CD11b,CD4,...,CD163,Bcatenin,CD138,Arginase1,CD73,CD206,MUC6,COX2,MMP9,cell_type
0,-0.015838,-0.262392,-0.328807,-0.030104,-0.169786,-0.195315,-0.144450,-0.314147,-0.306023,-0.681588,...,-0.189296,-0.395993,-0.229646,0.183429,-0.471492,0.368175,-0.137850,0.148542,-0.161118,Innate
1,-0.063117,-0.075990,1.852914,-0.030104,-0.169786,2.408519,-0.144450,1.413897,1.150271,-0.499367,...,-0.132587,5.211724,-0.229627,3.152803,-0.460310,0.406059,0.247144,0.549028,-0.114117,Innate
2,-0.070705,-0.262392,-0.362758,-0.030104,-0.169786,0.281066,-0.144449,-0.270863,-0.306020,-0.681588,...,-0.189296,-0.322642,-0.229654,-0.300006,0.798789,-0.238657,-0.137942,-0.167777,-0.161118,PDPN
3,-0.061208,-0.186271,-0.074156,-0.030104,-0.169786,-0.078391,-0.144447,-0.088802,0.294455,-0.681588,...,-0.188451,0.120267,-0.224748,1.068848,-0.256614,-0.230057,-0.130576,-0.069935,-0.161118,Endothelial
4,-0.070660,-0.123091,0.006811,-0.030104,0.100844,0.054867,-0.144450,-0.046886,2.337389,-0.588588,...,-0.189296,0.236230,-0.229654,3.108413,-0.451207,-0.165934,-0.126458,0.048613,-0.153662,Innate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173963,-0.041959,-0.126433,-0.283366,-0.021678,-0.163694,-0.147510,-0.124578,-0.107717,-0.263219,-0.432906,...,-0.184990,1.444015,-0.055955,-0.080908,-0.408275,-0.223434,-0.098247,-0.111970,-0.161118,Squamous_epithelial
173964,-0.047270,-0.012078,0.711355,-0.030104,-0.162891,-0.026921,-0.059392,-0.167543,-0.305407,-0.562331,...,-0.188423,2.483089,-0.210166,-0.187506,-0.308462,-0.225225,-0.120709,-0.121955,-0.161118,Squamous_epithelial
173965,-0.070705,-0.262392,-0.284593,-0.030104,-0.169786,-0.208046,-0.126339,-0.260718,-0.306023,-0.660122,...,-0.189296,0.402311,-0.030738,-0.076339,-0.396891,-0.236290,-0.135009,-0.132492,-0.161118,Squamous_epithelial
173966,-0.062836,1.355875,0.139548,-0.030104,-0.169739,-0.246346,-0.110626,-0.267340,-0.305779,-0.467839,...,-0.189296,1.024125,1.222364,-0.173762,-0.460017,-0.244242,-0.127680,0.124011,-0.161097,Squamous_epithelial


In [106]:
scaler = StandardScaler()

In [107]:
tonsil_filtered['cell_type'] = tonsil_filtered['cell_type'].astype('category')
be_filtered['cell_type'] = be_filtered['cell_type'].astype('category')

/var/folders/ry/jl4ll0zx1l709rh4r7_0cq6r0000gn/T/ipykernel_17834/3052414476.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tonsil_filtered['cell_type'] = tonsil_filtered['cell_type'].astype('category')
/var/folders/ry/jl4ll0zx1l709rh4r7_0cq6r0000gn/T/ipykernel_17834/3052414476.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  be_filtered['cell_type'] = be_filtered['cell_type'].astype('category')


In [108]:
X_train = tonsil_filtered.drop(columns=['cell_type'])
X_test = be_filtered.drop(columns=['cell_type'])
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train = tonsil_filtered['cell_type']
y_test = be_filtered['cell_type']

In [109]:
label_encoder = LabelEncoder()

# Fit and transform the training labels
y_train_encoded = label_encoder.fit_transform(y_train)

# Transform the test labels
y_test_encoded = label_encoder.transform(y_test)


In [110]:
xgb = XGBClassifier( n_thread = 8, max_depth=10, tree_method='auto', scale_pos_weight=1)

In [111]:
xgb.fit(X_train_scaled, y_train_encoded)

/Users/lukashat/miniforge3/envs/xgboost/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [14:41:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "n_thread", "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, n_thread=8,
              num_parallel_tree=None, ...)

In [112]:
ypred = xgb.predict(X_test_scaled)

In [113]:
accuracy = accuracy_score(y_test_encoded, ypred)

In [114]:
accuracy

0.8559749464315147

In [115]:
y_test_decoded = label_encoder.inverse_transform(y_test_encoded)
ypred_decoded = label_encoder.inverse_transform(ypred)

# Generate the classification report
report = classification_report(y_test_decoded, ypred_decoded)
print(report)

                     precision    recall  f1-score   support

        Endothelial       0.82      0.79      0.81      6181
             Innate       0.78      0.95      0.86      4282
              Nerve       0.92      0.55      0.69      2047
               PDPN       0.84      0.89      0.87       914
             Plasma       0.95      0.80      0.87      1177
       SmoothMuscle       0.91      0.92      0.91      9023
Squamous_epithelial       0.91      0.81      0.86      1077
             Stroma       0.82      0.83      0.83      4218
                  T       0.90      0.96      0.93      1416

           accuracy                           0.86     30335
          macro avg       0.87      0.84      0.85     30335
       weighted avg       0.86      0.86      0.85     30335

